# Double descent on GP datasets

Fixed training set size, a lot of different hidden layer sizes.

In [ ]:
import os
import sys
# If we don't need CUDA, do this before importing TF
os.environ["CUDA_VISIBLE_DEVICES"] = "-1"
import tensorflow as tf
import numpy as np
import pandas as pd
import tqdm
import tqdm.notebook
import scipy.stats
import matplotlib.pyplot as plt
import seaborn as sns
import IPython
import GPy
sns.set()

gpus = tf.config.experimental.list_physical_devices('GPU')
if gpus:
    tf.config.experimental.set_visible_devices([gpus[1]], 'GPU')

os.chdir("/nfs/scistore12/chlgrp/vvolhejn/smooth/logs/0314_double_descent/")

In [ ]:
%load_ext autoreload
%aimport smooth.datasets
%aimport smooth.model
%aimport smooth.analysis
%aimport smooth.callbacks
%aimport smooth.measures
%aimport smooth.util
%autoreload 1

In [ ]:
ms = pd.read_feather("./measures.feather")
print("Before removal:", len(ms))
# ms_nn = ms_nn.loc[np.isfinite(ms_nn["path_length_f_test"])]
ms = ms.loc[ms["error"].isnull()]
print("After removal:", len(ms))
smooth.analysis.remove_constant_columns(ms, verbose=True)
ms

In [ ]:
!cat ./run_config.yaml

In [ ]:
def plot_measures(ms1):
    measure_cols = [
        "loss_train", "loss_test",
        "gradient_norm",
        "path_length_f_test",
        "path_length_d_test",
        "weights_rms",
    ]

    for measure in measure_cols:
        IPython.display.display(IPython.display.Markdown("### {}".format(measure)))
        if True:
    #         ms1 = ms_mean[(ms_nn["hidden_size"] == 64)]
    #         ms1 = ms_nn[(ms_nn["init_scale"] == 10.)]
            grid = sns.relplot(
                data=ms1,
                x="hidden_size",
                y=measure,
                hue="dim",
    #             col_wrap=3,
                kind="line",
                palette=smooth.analysis.make_palette(ms1["dim"].unique()),
            )
            ax = grid.axes[0][0]

            ax.set_xscale("log")
            if "loss" in measure or True:
                ax.set_yscale("log")
            plt.show()

plot_measures(
    ms.loc[(ms["samples_train"] == 100) & (ms["dim"] == 128)]
)